In [53]:
import pandas as pd
import numpy as np
from sklearn import model_selection

In [54]:
df = pd.read_csv('clean_data.csv')
df.head()

,Адрес,Жилая,Кухня,Общая,Цена,Цена за квадрат,Этаж,Этажность здания,Время,Способ передвижения
0,"Московская область, Мытищи, мкр. Ярославский, ...",24.0,10.0,48.0,6550000.0,136458.0,12,25,15.0,на транспорте
1,"Москва, ЗАО, р-н Дорогомилово, Кутузовский про...",45.0,NaN,64.0,26000000.0,406250.0,4,7,13.0,пешком
2,"Москва, ЦАО, р-н Тверской, 1-й Колобовский пер...",23.4,8.9,40.0,18000000.0,450000.0,2,3,5.0,пешком
3,"Москва, ЗАО, р-н Раменки, Мосфильмовская ул., 8",63.0,20.0,98.0,59900000.0,611224.0,33,54,11.0,на транспорте
4,"Москва, ЦАО, р-н Хамовники, Комсомольский прос...",48.0,NaN,67.3,37900000.0,563150.0,4,5,10.0,пешком


In [67]:
df= df.drop(columns='Адрес', axis=1)

In [73]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Способ передвижения'] = le.fit_transform(df['Способ передвижения'])

In [74]:
df_train, df_test = model_selection.train_test_split(df, test_size=0.33, random_state=42)

In [75]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [76]:
imputer.fit(df_train)
df_train_full = imputer.transform(df_train)
df_test_full = imputer.transform(df_test)